## Setting up 

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, LabelSet
import numpy as np

output_notebook()

Loading BokehJS ...

## Defining Variables and Functions

In [2]:
# long string variables
twentysix = [1,2,3,4,5,6,7,8,9,10,11,12,
             13,14,15,16,17,18,19,20,21,22,23,24,25,26]
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
            'n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = alphabet*3

# rotor shift
shift = 10

# aesthetic variables
x_offset = -4.5    # letter annotation offset from circle edge
y_offset = -7    # letter annotation offset from circle edge

seg_color = 'gold'    # segment color
line_width = 3.0    # segment line width
seg_alpha = 0.6    # segment alpha

circ_color = 'gold'    # circle color
circ_size = 23    # circle size
circ_alpha = 0.4    # circle alpha
hov_color = 'gold'    # hover color for circles
hov_alpha = 0.8    # hover alpha for circles

font = 'Garamond'


# some points
x = np.array([43,48,50])
x = np.repeat(x,26)
y = twentysix*3

In [3]:
# shifting functions

def shift_yvals(yvals, shift):
    for pair in yvals:
        pair[1] += shift
        if pair[1] > 26:
            pair[1] += (-26)
    return yvals

def shift_links(links,shift):
    for item in links:
        if item > 51:
            links[item][0] += (- shift)
            if links[item][0] < 26:
                links[item][0] += 26
        if item > 25 and item < 52:
            links[item][1] += shift
            if links[item][1] > 77:
                links[item][1] += (- 26)
    return(links)

## Creating X and Y values for paths

In [4]:
# X values
xval1 = [[43,48]]*26
xval2 = [[48,50]]*26
xvals = xval1+xval2

In [5]:
# Y values
# rotor 7
yvals_inner = [[1,14],[2,26],[3,10],[4,8],[5,7],[6,18],[7,3],[8,24],
    [9,13],[10,25],[11,19],[12,23],[13,2],[14,15],[15,21],[16,6],[17,1],
    [18,9],[19,22],[20,12],[21,16],[22,5],[23,11],[24,17],[25,4],
    [26,20]]
yvals_outer = [[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],
    [9,9],[10,10],[11,11],[12,12],[13,13],[14,14],[15,15],[16,16],
    [17,17],[18,18],[19,19],[20,20],[21,21],[22,22],[23,23],[24,24],
    [25,25],[26,26]]

yvals_outer = shift_yvals(yvals_outer, shift)
yvals = yvals_inner + yvals_outer

# Rotor Links - Rotor 7 [elephant]

In [6]:
# establish links
links = {
    0:[39],     1:[51],      2:[35],     3:[33],     4:[32],     5:[43],      
    6:[28],     7:[49],      8:[38],     9:[50],    10:[44],    11:[48],   
   12:[27],    13:[40],     14:[46],    15:[31],    16:[26],    17:[34],    
   18:[47],    19:[37],     20:[41],    21:[30],    22:[36],    23:[42],    
   24:[29],    25:[45], 
        
   26:[16,52],    27:[12,53],     28:[6,54],     29:[24,55],    
   30:[21,56],    31:[15,57],     32:[4,58],     33:[3,59],      
   34:[17,60],    35:[2,61],      36:[22,62],    37:[19,63],
   38:[8,64],     39:[0,65],      40:[13,66],    41:[20,67],    
   42:[23,68],    43:[5,69],      44:[10,70],    45:[25,71],     
   46:[14,72],    47:[18,73],     48:[11,74],    49:[7,75],  
   50:[9,76],     51:[1,77],
    
   52:[26],    53:[27],    54:[28],    55:[29],    56:[30],    57:[31],
   58:[32],    59:[33],    60:[34],    61:[35],    62:[36],    63:[37],
   64:[38],    65:[39],    66:[40],    67:[41],    68:[42],    69:[43],
   70:[44],    71:[45],    72:[46],    73:[47],    74:[48],    75:[49],
   76:[50],    77:[51]
}

links = shift_links(links,shift)

## Create Graph and Add Features

In [7]:
# create graph
p = figure(width=400, height=800, tools="",
           toolbar_location=None,title='Rotor 7')

# add letter labels
lsource = ColumnDataSource(data=dict(x = x, y = y, names = alphabet))
labels = LabelSet(x='x', y='y', text='names', level='annotation',
                  x_offset=x_offset, y_offset=y_offset, source=lsource,
                  render_mode='canvas', text_font=font, 
                  text_color='black', text_font_style='bold')
p.add_layout(labels)

# add hover glyphs
source = ColumnDataSource({'x0': [], 'y0': [], 'x1': [], 'y1':[]})
sr = p.segment(x0='x0',y0='y0',x1='x1', y1='y1', color=seg_color, 
               alpha=seg_alpha, line_width=line_width, source=source,)
cr = p.circle(x, y, color=circ_color, size=circ_size, alpha=circ_alpha, 
              hover_color=hov_color, hover_alpha=hov_alpha)

# add path glyphs
p.multi_line(xvals,yvals,color='gray',alpha=0.3)

# remove gridlines and axis labels
p.xgrid.visible = False
p.ygrid.visible = False
p.axis.visible = False

# JS for hover tool
code = """
var links = %s;
var data = {'x0': [], 'y0': [], 'x1': [], 'y1': []};
var cdata = circle.get('data');
var indices = cb_data.index['1d'].indices;
for (i=0; i < indices.length; i++) {
    ind0 = indices[i]
    for (j=0; j < links[ind0].length; j++) {
        ind1 = links[ind0][j];
        data['x0'].push(cdata.x[ind0]);
        data['y0'].push(cdata.y[ind0]);
        data['x1'].push(cdata.x[ind1]);
        data['y1'].push(cdata.y[ind1]);
    }
}
segment.set('data', data);
""" % links

# Call JS and Add HoverTool
callback = CustomJS(args={'circle':cr.data_source, 
                          'segment':sr.data_source}, code=code)
p.add_tools(HoverTool(tooltips=None,callback=callback,renderers=[cr]))

# Display Graph

In [8]:
show(p)